In [113]:
import random
import numpy as np

In [114]:
# input of size 74 x 74 x 32 in 16 bit float

# Array of random numbers between -100 to 100
inp = np.random.uniform(low = -10,
                           high = 10,
                           size = (32,74,74) #size = (chn,rows,cols)
                           ).astype(np.float16)


ker = np.random.uniform(low = -4,
                           high = 4,
                           size = (32,3,3) #size = (chn,rows,cols)
                           ).astype(np.float16)




In [115]:
out = np.zeros((72,72), dtype=np.float16)
out_buffer = np.zeros((36,), dtype=np.float32)

In [116]:
def mac(inp,ker):
  acc = 0
  num_cyl = len(inp)
  num_chn = len(inp[0])
  for cyl in range(num_cyl):
    for chn in range(num_chn):
      acc += (inp[cyl][chn]*ker[cyl][chn])
  return acc

In [117]:
row = 0
partitions = 72 // 36
chn = 0
part = 0


while(row<72):
  part = 0
  while(part<partitions):
    out_buffer = np.zeros((36,),dtype=np.float32)
    start_col = part*36
    end_col = (part+1)*36
    # ----------------------------- Computation of 36 final output values -----------------------------------
    chn = 0
    while(chn<32):

      # Kernel cylinders will be fixed for 1 iteration of partial output calculation
      kcyl1 = np.array([ker[chn+i][0][0] for i in range(8)]).astype(np.float32)
      kcyl2 = np.array([ker[chn+i][0][1] for i in range(8)]).astype(np.float32)
      kcyl3 = np.array([ker[chn+i][0][2] for i in range(8)]).astype(np.float32)

      kcyl4 = np.array([ker[chn+i][1][0] for i in range(8)]).astype(np.float32)
      kcyl5 = np.array([ker[chn+i][1][1] for i in range(8)]).astype(np.float32)
      kcyl6 = np.array([ker[chn+i][1][2] for i in range(8)]).astype(np.float32)

      kcyl7 = np.array([ker[chn+i][2][0] for i in range(8)]).astype(np.float32)
      kcyl8 = np.array([ker[chn+i][2][1] for i in range(8)]).astype(np.float32)
      kcyl9 = np.array([ker[chn+i][2][2] for i in range(8)]).astype(np.float32)

      kernel_cylinders = [kcyl1, kcyl2, kcyl3,
                          kcyl4, kcyl5, kcyl6,
                          kcyl7, kcyl8, kcyl9
                          ]
      # ----------------------------- Computation of 36 partial output values -----------------------------------
      col = start_col
      while(col < end_col):
        ker_pos = 0
        # ----------------------- Convolution between input and kernels -----------------------------
        while(ker_pos<6): # Unroll the following loop in final implementation
        # Input cylinders
          incyl1 = np.array([inp[chn+i][row][(col)+ker_pos] for i in range(8)]).astype(np.float32)
          incyl2 = np.array([inp[chn+i][row][(col+1)+ker_pos] for i in range(8)]).astype(np.float32)
          incyl3 = np.array([inp[chn+i][row][(col+2)+ker_pos] for i in range(8)]).astype(np.float32)

          incyl4 = np.array([inp[chn+i][row+1][(col)+ker_pos] for i in range(8)]).astype(np.float32)
          incyl5 = np.array([inp[chn+i][row+1][(col+1)+ker_pos] for i in range(8)]).astype(np.float32)
          incyl6 = np.array([inp[chn+i][row+1][(col+2)+ker_pos] for i in range(8)]).astype(np.float32)

          incyl7 = np.array([inp[chn+i][row+2][(col)+ker_pos] for i in range(8)]).astype(np.float32)
          incyl8 = np.array([inp[chn+i][row+2][(col+1)+ker_pos] for i in range(8)]).astype(np.float32)
          incyl9 = np.array([inp[chn+i][row+2][(col+2)+ker_pos] for i in range(8)]).astype(np.float32)

          input_cylinders = [incyl1, incyl2, incyl3,
                              incyl4, incyl5, incyl6,
                              incyl7, incyl8, incyl9
                              ]

          dot_prod = mac(input_cylinders,kernel_cylinders)
          out_buffer[(col+ker_pos)%36] += dot_prod

          ker_pos+=1
        col+=6
        # ----------------------------- End of Computation of 36 partial output values -----------------------------------
      chn+=8
    # ----------------------------- End of computation of 36 final output values -----------------------------------
    # print(out_buffer)
    for ind in range(36):
      out[row][start_col+ind] = out_buffer[ind]
    part+=1
  # print(row)
  row+=1

# print(out)
our_impl_tensor = torch.from_numpy(out).half()
print(our_impl_tensor)

tensor([[ -25.7656, -167.5000,  130.7500,  ...,  -10.5312, -102.5000,
         -106.0000],
        [  20.4531,  -52.3125, -127.3750,  ...,  102.6250,  -72.8125,
          113.4375],
        [-104.4375, -533.0000,    6.9414,  ...,  -70.8125,  -88.0000,
         -321.2500],
        ...,
        [  91.4375,  125.7500,  280.2500,  ..., -129.8750, -146.6250,
         -151.2500],
        [  79.8750,  -42.4688,    8.9609,  ..., -282.5000,  -28.4375,
          225.5000],
        [-134.5000, -241.5000,  151.0000,  ..., -126.4375, -202.5000,
          108.0625]], dtype=torch.float16)


In [118]:
import torch
import torch.nn.functional as F

new_inp = np.expand_dims(inp,axis=0)
new_ker = np.expand_dims(ker,axis=0)

inp_tensor = torch.from_numpy(new_inp.astype(np.float32))
ker_tensor = torch.from_numpy(new_ker.astype(np.float32))
tensor_out = F.conv2d(input=inp_tensor,weight=ker_tensor)

torch_imp_tensor = tensor_out.half()[0][0]
print(torch_imp_tensor)

tensor([[ -25.7656, -167.5000,  130.7500,  ...,  -10.5312, -102.5000,
         -106.0000],
        [  20.4531,  -52.3125, -127.3750,  ...,  102.6250,  -72.8125,
          113.4375],
        [-104.4375, -533.0000,    6.9414,  ...,  -70.8125,  -88.0000,
         -321.2500],
        ...,
        [  91.4375,  125.7500,  280.2500,  ..., -129.8750, -146.6250,
         -151.2500],
        [  79.8750,  -42.4688,    8.9609,  ..., -282.5000,  -28.4375,
          225.5000],
        [-134.5000, -241.5000,  151.0000,  ..., -126.4375, -202.5000,
          108.0625]], dtype=torch.float16)


In [119]:
diff = torch.sub(our_impl_tensor,torch_imp_tensor)
print("Max difference between our implementation and torch implementation ",torch.max(diff))
print("Min difference between our implementation and torch implementation ",torch.min(diff))

Max difference between our implementation and torch implementation  tensor(0.0010, dtype=torch.float16)
Min difference between our implementation and torch implementation  tensor(0., dtype=torch.float16)


In [120]:

diff_indices = torch.nonzero(diff)
for indi in diff_indices:
  row = indi[0]
  col = indi[1]
  print("Value I got ",our_impl_tensor[row][col])
  print("Value pytorch got ",torch_imp_tensor[row][col])
  print("Difference Value is ",diff[row][col])
  meas = our_impl_tensor[row][col]
  real = torch_imp_tensor[row][col].numpy()
  error = abs(diff[row][col].numpy())
  print("Relative error is ", abs(error/real)*100)
  print()

Value I got  tensor(-0.1246, dtype=torch.float16)
Value pytorch got  tensor(-0.1247, dtype=torch.float16)
Difference Value is  tensor(6.1035e-05, dtype=torch.float16)
Relative error is  0.04897117614746094

Value I got  tensor(0.2161, dtype=torch.float16)
Value pytorch got  tensor(0.2159, dtype=torch.float16)
Difference Value is  tensor(0.0001, dtype=torch.float16)
Relative error is  0.056552886962890625

Value I got  tensor(-1.5879, dtype=torch.float16)
Value pytorch got  tensor(-1.5889, dtype=torch.float16)
Difference Value is  tensor(0.0010, dtype=torch.float16)
Relative error is  0.06146430969238281



In [121]:
# print(torch.sub(new_out,tor_out)[0])

In [122]:
#CODE FOR CONVOLUTION
# Jitendra Implementation
#size of an image channel : m*n
#no of channels: c
#size of kernel: k*k
#size of output buffer: buff
#no of parallel MACs: p (3 here)
#assuming image is padded with zeros

def MAC(img,kernel,i,j,k,ch):
  #c for number of channels, here 8
  output = 0
  for i in range(k):
    for j in range(k):
      for c in range(ch):
        output += img[c][i][j]*kernel[c][i][j]
  return output

def convolution(image,kernel):
  m=len(image[0])
  n=len(image[0][0])
  c=len(image)
  print(c)
  k=len(kernel[0])
  p=3
  x = int(m/k)
  y = int(n/k)
  out_buff = [[0]*y for z in range(x)]
  for ch in range(int(c/8)):
    i=0
    j=0
    while (i<=m-k):
      out_buff[int(i/k)][int(j/k)] += MAC(image,kernel,i,j,k,c)
      j += k
      if j==n :
        j = 0
        i += k
      if i>m-k:
        break
      out_buff[int(i/k)][int(j/k)] += MAC(image,kernel,i,j,k,c)
      j += k
      if j==n :
        j = 0
        i += k
      if i>m-k:
        break
      out_buff[int(i/k)][int(j/k)] += MAC(image,kernel,i,j,k,c)
      j += k
      if j==n :
        j = 0
        i += k
      if i>m-k:
        break
  print(out_buff)

image = [[[1]*12 for i in range(9)]]*8
kernel = [[[1]*3 for i in range(3)]]*8
convolution(image,kernel)

8
[[72, 72, 72, 72], [72, 72, 72, 72], [72, 72, 72, 72]]
